In [1]:
import pandas as pd
import glob
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandoc

# pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.precision = 4

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [7]:
# list the files pathes:
# pthdct = {'g2d_MC03':r"C:\SVN\MonoCalibrationG2D_VAVE_ini\MC03_Mono.binningcfg.xml",
#          'g2e_MC03':r"C:\SVN\MonoCalibrationG2E_VAVE_ini\MC03_Mono.binningcfg.xml",
#          'g1_MC03':r"C:\SVN\MonoCalibrationG1_VAVE_ini\MC03_Mono.binningcfg.xml"
# }

pthroot = r"C:\work\SVN\PMCal\SEN5X_Calibration_ini"
# pthroot = r"C:\SVN\SEN5XLaserModuleCalibration_ini\3.000.667-001"

# MC Parser

In [8]:
class xml_parser:
    def __init__(self,pthroot,MC):
        self.MC = MC
        if isinstance(pthroot,str):
            filepath = []
            for root,dirs,files in os.walk(pthroot):
                for file in files:
                    filepath.append(os.path.join(root,file))
            if [f for f in filepath if all(x in f for x in [MC,"binningcfg"])]:
                self.path = [f for f in filepath if all(x in f for x in [MC,"binningcfg"])]
            else:
                print("No {} files in the path".format(MC))
        elif isinstance(pthroot,dict):
            self.path = pthroot
        else:
            print("Input as str or dict")
        
    def xml_mc(self):
        testname = pd.DataFrame()
        if isinstance(self.path,list):
            for pth in self.path:
                d = pd.read_xml(pth,xpath=r".//{}".format(self.MC))
                phaselt = d.columns

                setupxml = pd.DataFrame()
                for ph in phaselt:
                    d = pd.read_xml(pth,xpath=r".//{}/{}".format(self.MC,ph))
                    tid = [x for x in d.columns if x.startswith("T")]

                    for id in tid:
                        d = pd.read_xml(pth,xpath=r".//{}/{}/{}".format(self.MC,ph,id))
                        d['Phase'] = ph
                        d['TestId'] = id
                        setupxml = pd.concat([setupxml,d])

                setupxml['Setup'] = os.path.basename(pth)[0:28]
                testname = pd.concat([testname,setupxml])

            testname['item'] = testname['TestId'] + '_' + testname['Phase'] + '_' + testname['TestName']
            testname = testname.replace('none',np.nan)
            try:
                testname = testname[testname['item'].map(lambda x: False if all(y in x for y in ['Ratio','SPS']) else True)]
            except:
                pass
            return testname

        elif isinstance(self.path,dict):
            for key in self.path:
                pth = pthdct[key]
                d = pd.read_xml(pth,xpath=r".//{}".format(self.MC))
                phaselt = d.columns

                setupxml = pd.DataFrame()
                for ph in phaselt:
                    d = pd.read_xml(pth,xpath=r".//{}/{}".format(self.MC,ph))
                    tid = [x for x in d.columns if x.startswith("T")]

                    for id in tid:
                        d = pd.read_xml(pth,xpath=r".//{}/{}/{}".format(self.MC,ph,id))
                        d['Phase'] = ph
                        d['TestId'] = id
                        setupxml = pd.concat([setupxml,d])

                setupxml['Setup'] = key
                testname = pd.concat([testname,setupxml])

            testname['item'] = testname['TestId'] + '_' + testname['Phase'] + '_' + testname['TestName']
            testname = testname.replace('none',np.nan)
            try:
                testname = testname[testname['item'].map(lambda x: False if all(y in x for y in ['Ratio','SPS']) else True)]
            except:
                pass
            return testname
        else:
            print("Input the dict or list, please!")
            
    def xml_mc_plot(self):
        testname = self.xml_mc()
        for item,dt in testname.groupby('item'):
            ax = dt.plot.line(x='Setup',
                           y=['UTL','LTL'],
                           style='.',
                            figsize=(32,8),
                            fontsize=14,
                            markersize=18,
                            rot = 90,
                        )
            xt = ax.set_xticks(list(range(dt.shape[0])))
            xtlab = ax.set_xticklabels(dt['Setup'])
            xtitle = ax.set_title(item)
            plt.show()
            
    def xml_mcdiff_plot(self):
        testname = self.xml_mc()
        ch = []
        for item,dt in testname.groupby("item"):
            if any([len(dt["LTL"].unique()) > 1,len(dt["UTL"].unique()) > 1]):
                ax = dt.plot.line(x='Setup',
                       y=['UTL','LTL'],
                       style='.',
                        figsize=(32,8),
                        fontsize=14,
                        markersize=18,
                        rot = 90,
                    )
                xt = ax.set_xticks(list(range(dt.shape[0])))
                xtlab = ax.set_xticklabels(dt['Setup'])
                xtitle = ax.set_title(item)
                plt.show()
            else:
                ch.append(False)
        if not all(ch):
            print("No different parameters!")
            
    def binmapping(self):

        hardbinmapping = pd.DataFrame()
        if isinstance(self.path,dict):
            for key in self.path:
                pth = pthdct[key]
                d = pd.read_xml(pth,xpath=r".//SoftBins//*")
                d['SetupRecipe'] = key
                hardbinmapping = pd.concat([hardbinmapping,d])
            return hardbinmapping
        elif isinstance(self.path,list):
            # setupxml = pd.DataFrame()
            for pth in self.path:
                setup = os.path.basename(pth)[0:28]
                d = pd.read_xml(pth,xpath=r".//SoftBins//*")
                d['SetupRecipe'] = setup
                hardbinmapping = pd.concat([hardbinmapping,d])
            return hardbinmapping
        else:
            print("Input the dict or list, please!")   
            
    def binmapping_plot(self):
        hdmp = self.binmapping()
        for sftb,gp in hdmp.groupby("BinCode"):
            ax = gp.plot.line(style='.',
                        x='SetupRecipe',
                        y='HardBinCode',
                        title=sftb,
                        markersize=14,
                        figsize=(14,4),
                        rot=90,
                             )
            xt = ax.set_xticks(list(range(gp.shape[0])))
            xtlab = ax.set_xticklabels(gp['SetupRecipe'])
            # xtitle = ax.set_title(item)
            plt.show()

# Initialization

In [9]:
# xml_parser 可以接受列标或字典的地址形式:
parser = xml_parser(pthroot,MC="MC03")

# binmapping: SoftBin-HardBin

In [10]:
bins = parser.binmapping()

In [11]:
bins.head(2)

,BinCode,BinQuality,BinDescription,TurnChannelOff,OverrideFailBins,HardBinCode,SetupRecipe
0,0x00,SystemError,SystemError,True,True,3,MC03_CN-100340_3.000.535-007
1,0x70,Incomplete,Incomplete,True,True,4,MC03_CN-100340_3.000.535-007


In [7]:
bins.query("HardBinCode==7")

,BinCode,BinQuality,BinDescription,TurnChannelOff,HardBinCode,SetupRecipe


In [27]:
blt = bins[bins["HardBinCode"].isin([4,10])]["BinCode"].unique()
blt

array(['0x70', '0x72', '0x31', '0x7F', '0xE3', '0xE4', '0xE5', '0xF1',
       '0xF2', '0xFA', '0xFB', '0xFC', '0x75', '0xE0', '0x5F'],
      dtype=object)

# binmapping Plot

In [ ]:
# HardBin_SoftBin Mappling plot:
parser.binmapping_plot()

# binning table

## MC03

In [12]:
# xml_parser 可以接受列标或字典的地址形式:
parser = xml_parser(pthroot,MC="MC03")
mc = parser.xml_mc()
# mc03.head(50)

In [13]:
mc.head(2)

,TestName,Unit,LBin,UBin,LTL,UTL,Comment,Phase,TestId,Setup,item
0,AtmHeatingDeviceCurrent,mA,0xFA,0xFA,130.0,140.0,NaN,PmPeriphery,T150011,MC03_CN-100340_3.000.535-007,T150011_PmPeriphery_AtmHeatingDeviceCurrent
0,T1RefMean,DegC,0xFC,0xFC,22.0,28.0,NaN,GetAmbientEnvironment,T020001,MC03_CN-100340_3.000.535-007,T020001_GetAmbientEnvironment_T1RefMean


In [13]:
mc[mc["TestName"].map(lambda x: True if "ZeroInterceptPm10p0MeanPartMassRef" in x else False) 
   & mc["Setup"].map(lambda x: True if "MC03_CN-100340_3.000.228-004" in x else False) 
    ].sort_values("Phase")

,TestName,Unit,LBin,UBin,LTL,UTL,Comment,Phase,TestId,Setup,item
0,ZeroInterceptPm10p0MeanPartMassRef,pg,0x75,0x75,0.8704,1.7804,NaN,Aerosol_Arz,T006006,MC03_CN-100340_3.000.228-004,T006006_Aerosol_Arz_ZeroInterceptPm10p0MeanPar...
0,ZeroInterceptPm10p0MeanPartMassRef,pg,0x75,0x75,0.0284,0.1234,NaN,Aerosol_Kcl,T006006,MC03_CN-100340_3.000.228-004,T006006_Aerosol_Kcl_ZeroInterceptPm10p0MeanPar...
0,ZeroInterceptPm10p0MeanPartMassRef,pg,0x75,0x75,NaN,NaN,NaN,Aerosol_Remeasurement,T006006,MC03_CN-100340_3.000.228-004,T006006_Aerosol_Remeasurement_ZeroInterceptPm1...


## MQ

In [54]:
parsersen = xml_parser(pthroot,MC="MQ01")
mq = parsersen.xml_mc()

In [55]:
mq[mq["TestName"].map(lambda x: True if "Vcc" in x else False) 
  | mq["TestName"].map(lambda x: True if "VoltageShieldTest" in x else False) 
   & mq["Setup"].map(lambda x: True if "3.000.540-003" in x else False) 
    ].sort_values("Phase")

,TestName,Unit,LBin,UBin,LTL,UTL,Comment,Phase,TestId,Setup,item
0,VoltageShieldTest,V,0x30,0x70,-0.3,0.3,added by BinningCfgPatcher,SENXXDiodeMeasurement,T031006,MQ01_CN-100340_3.000.540-003,T031006_SENXXDiodeMeasurement_VoltageShieldTest
0,VoltageShieldTest,V,0x30,0x70,-0.3,0.3,added by BinningCfgPatcher,SENXXDiodeMeasurement,T031006,MQ01_CN-100341_3.000.540-003,T031006_SENXXDiodeMeasurement_VoltageShieldTest
0,VoltageShieldTest,V,0x30,0x70,-0.3,0.3,added by BinningCfgPatcher,SENXXDiodeMeasurement,T031006,MQ01_CN-100342_3.000.540-003,T031006_SENXXDiodeMeasurement_VoltageShieldTest
0,VoltageShieldTest,V,0x30,0x70,-0.3,0.3,added by BinningCfgPatcher,SENXXDiodeMeasurement,T031006,MQ01_CN-100343_3.000.540-003,T031006_SENXXDiodeMeasurement_VoltageShieldTest


# binningdiff plot

In [ ]:
parser.xml_mcdiff_plot()

# binning plot

In [ ]:
parser.xml_mc_plot()